# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 11:19AM,237855,19,60016022,"GUSA Granules USA, Inc.",Released
1,May 23 2022 11:17AM,237854,12,SO1012060,Twinlab Consolidated Corporation,Released
2,May 23 2022 10:56AM,237853,10,CTF0011252,"Citieffe, Inc.",Released
3,May 23 2022 10:40AM,237852,19,60014867,"GUSA Granules USA, Inc.",Released
4,May 23 2022 10:35AM,237851,10,Enova-10002,Emerginnova,Released
5,May 23 2022 10:35AM,237851,10,1-10000,Emerginnova,Released
6,May 23 2022 10:32AM,237850,20,8051134,"Exact-Rx, Inc.",Released
7,May 23 2022 10:32AM,237850,20,8052358,"Exact-Rx, Inc.",Released
8,May 23 2022 10:31AM,237849,10,8046556,Eywa Pharma Inc.,Released
9,May 23 2022 10:31AM,237849,10,8051695,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,237851,Released,2
32,237852,Released,1
33,237853,Released,1
34,237854,Released,1
35,237855,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237851,NaN,NaN,2.0
237852,NaN,NaN,1.0
237853,NaN,NaN,1.0
237854,NaN,NaN,1.0
237855,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237781,0.0,0.0,51.0
237789,3.0,4.0,0.0
237792,0.0,13.0,5.0
237794,0.0,0.0,26.0
237795,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237781,0,0,51
237789,3,4,0
237792,0,13,5
237794,0,0,26
237795,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237781,0,0,51
1,237789,3,4,0
2,237792,0,13,5
3,237794,0,0,26
4,237795,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237781,,,51
1,237789,3,4,
2,237792,,13,5
3,237794,,,26
4,237795,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 11:19AM,237855,19,"GUSA Granules USA, Inc."
1,May 23 2022 11:17AM,237854,12,Twinlab Consolidated Corporation
2,May 23 2022 10:56AM,237853,10,"Citieffe, Inc."
3,May 23 2022 10:40AM,237852,19,"GUSA Granules USA, Inc."
4,May 23 2022 10:35AM,237851,10,Emerginnova
6,May 23 2022 10:32AM,237850,20,"Exact-Rx, Inc."
8,May 23 2022 10:31AM,237849,10,Eywa Pharma Inc.
29,May 23 2022 10:25AM,237844,20,Alumier Labs Inc.
87,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc."
88,May 23 2022 10:13AM,237841,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 11:19AM,237855,19,"GUSA Granules USA, Inc.",,,1
1,May 23 2022 11:17AM,237854,12,Twinlab Consolidated Corporation,,,1
2,May 23 2022 10:56AM,237853,10,"Citieffe, Inc.",,,1
3,May 23 2022 10:40AM,237852,19,"GUSA Granules USA, Inc.",,,1
4,May 23 2022 10:35AM,237851,10,Emerginnova,,,2
5,May 23 2022 10:32AM,237850,20,"Exact-Rx, Inc.",,,2
6,May 23 2022 10:31AM,237849,10,Eywa Pharma Inc.,,1,20
7,May 23 2022 10:25AM,237844,20,Alumier Labs Inc.,,11,47
8,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc.",,,1
9,May 23 2022 10:13AM,237841,10,ISDIN Corporation,,,58


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 11:19AM,237855,19,"GUSA Granules USA, Inc.",1,,
1,May 23 2022 11:17AM,237854,12,Twinlab Consolidated Corporation,1,,
2,May 23 2022 10:56AM,237853,10,"Citieffe, Inc.",1,,
3,May 23 2022 10:40AM,237852,19,"GUSA Granules USA, Inc.",1,,
4,May 23 2022 10:35AM,237851,10,Emerginnova,2,,
5,May 23 2022 10:32AM,237850,20,"Exact-Rx, Inc.",2,,
6,May 23 2022 10:31AM,237849,10,Eywa Pharma Inc.,20,1,
7,May 23 2022 10:25AM,237844,20,Alumier Labs Inc.,47,11,
8,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc.",1,,
9,May 23 2022 10:13AM,237841,10,ISDIN Corporation,58,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 11:19AM,237855,19,"GUSA Granules USA, Inc.",1,,
1,May 23 2022 11:17AM,237854,12,Twinlab Consolidated Corporation,1,,
2,May 23 2022 10:56AM,237853,10,"Citieffe, Inc.",1,,
3,May 23 2022 10:40AM,237852,19,"GUSA Granules USA, Inc.",1,,
4,May 23 2022 10:35AM,237851,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 11:19AM,237855,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,May 23 2022 11:17AM,237854,12,Twinlab Consolidated Corporation,1.0,NaN,NaN
2,May 23 2022 10:56AM,237853,10,"Citieffe, Inc.",1.0,NaN,NaN
3,May 23 2022 10:40AM,237852,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,May 23 2022 10:35AM,237851,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 11:19AM,237855,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,May 23 2022 11:17AM,237854,12,Twinlab Consolidated Corporation,1.0,0.0,0.0
2,May 23 2022 10:56AM,237853,10,"Citieffe, Inc.",1.0,0.0,0.0
3,May 23 2022 10:40AM,237852,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,May 23 2022 10:35AM,237851,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2616094,342.0,14.0,0.0
12,475694,2.0,0.0,0.0
15,951199,60.0,14.0,0.0
18,1426868,6.0,0.0,0.0
19,1189175,4.0,4.0,3.0
20,475694,49.0,11.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2616094,342.0,14.0,0.0
1,12,475694,2.0,0.0,0.0
2,15,951199,60.0,14.0,0.0
3,18,1426868,6.0,0.0,0.0
4,19,1189175,4.0,4.0,3.0
5,20,475694,49.0,11.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,342.0,14.0,0.0
1,12,2.0,0.0,0.0
2,15,60.0,14.0,0.0
3,18,6.0,0.0,0.0
4,19,4.0,4.0,3.0
5,20,49.0,11.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,342.0
1,12,Released,2.0
2,15,Released,60.0
3,18,Released,6.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,3.0,0.0
Executing,14.0,0.0,14.0,0.0,4.0,11.0
Released,342.0,2.0,60.0,6.0,4.0,49.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,14.0,0.0,14.0,0.0,4.0,11.0
2,Released,342.0,2.0,60.0,6.0,4.0,49.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,14.0,0.0,14.0,0.0,4.0,11.0
2,Released,342.0,2.0,60.0,6.0,4.0,49.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()